In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=2)
today, yesterday

(datetime.date(2022, 1, 29), datetime.date(2022, 1, 27))

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 1, 28), datetime.date(2022, 1, 21))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [3]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

3260

In [4]:
sql = """
SELECT name
FROM stocks 
WHERE status IN ("B","I","O","S") 
ORDER BY status, name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'EPG', 'GLOBAL', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'SCC', 'STA', 'SYNEX', 'BGRIM', 'CKP', 'IVL', 'LPF', 'MCS', 'NER', 'RJH', 'TOP', 'TSE', 'TSTH', 'TU', 'WHART', 'BBL', 'BGC', 'KBANK', 'KTC', 'PTTEP', 'ROJNA', 'SCB', 'TVO'"

### Get past one year data

In [5]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-01-27' AND name IN ('EPG', 'GLOBAL', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'SCC', 'STA', 'SYNEX', 'BGRIM', 'CKP', 'IVL', 'LPF', 'MCS', 'NER', 'RJH', 'TOP', 'TSE', 'TSTH', 'TU', 'WHART', 'BBL', 'BGC', 'KBANK', 'KTC', 'PTTEP', 'ROJNA', 'SCB', 'TVO') 
ORDER BY name, date


In [6]:
df = pd.read_sql(sql, const)
df.shape

(7284, 7)

In [7]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [8]:
sql = """
SELECT name,status 
FROM stocks 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
stocks = pd.read_sql(sql, conlite)
stocks.set_index(['name'],inplace=True)
stocks

,status
name,
BBL,O
BGC,O
BGRIM,I
CKP,I
EPG,B
GLOBAL,B
IVL,I
KBANK,O
KTC,O


### Create monitors from stocks

In [9]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,status
name,
BBL,O
BGC,O
BGRIM,I
CKP,I
EPG,B
GLOBAL,B
IVL,I
KBANK,O
KTC,O


In [10]:
monitors.shape[0]

30

In [11]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)

In [12]:
sql = """
SELECT status, COUNT(*) AS items 
FROM stocks 
WHERE name IN (%s)
GROUP BY status
ORDER BY status
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,status,items
0,B,10
1,I,12
2,O,8


In [13]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [14]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(119, 1)

In [15]:
df_merge = pd.merge(monitors, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(119, 3)

In [16]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,status,_merge


In [17]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(89, 3)

In [18]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
EPG,B,SET100
GLOBAL,B,SET50
NOBLE,B,SET
PTT,B,SET50
PTTGC,B,SET50
RATCH,B,SET50
RCL,B,SET100
SCC,B,SET50
STA,B,SET100


In [19]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [20]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BBL,O
BGC,O
EPG,B
GLOBAL,B
KBANK,O
KTC,O
NOBLE,B
PTT,B
PTTEP,O


In [21]:
buy_candidates.shape[0]

18

In [22]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BGRIM,I
CKP,I
IVL,I
LPF,I
MCS,I
NER,I
RJH,I
TOP,I
TSE,I


In [23]:
sell_candidates.shape[0]

12